# 1. Library, Data

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import squidpy as sq
from squidpy.im import ImageContainer
import pickle
import os 
import seaborn as sns
import matplotlib.colors
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import stlearn as st
from anndata import AnnData
from sklearn.model_selection import (train_test_split,) 
import tensorflow as tf
from tensorflow.keras.layers.experimental import (preprocessing,)
sc.settings.verbosity=3
sc.settings.set_figure_params(dpi=100, fontsize=10, dpi_save=600, format='tiff',)
path = '/path/data/brca'


In [ ]:
save_file=path+"/object/adata.h5ad"
adata=sc.read_h5ad(save_file)

In [ ]:
df = pd.read_csv(path+'tumor_border.csv')
vertices = df[['X', 'Y']].values.tolist() 
from matplotlib.path import Path
boundary = Path(vertices)
coords = adata.obsm['spatial']
is_inside = [boundary.contains_point(coord) for coord in coords]

# 2. Tumor

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from shapely.geometry import Polygon

# Coordinates of the polygon (including empty coordinates)
polygon_coords = vertices

# Remove empty coordinates to get valid coordinates
valid_coords = [coord for coord in polygon_coords if coord is not None]

# Separate the valid coordinates into x and y values
x_coords, y_coords = zip(*valid_coords)

# Create an interpolation function
interp_func = interp1d(x_coords, y_coords, kind='linear')

# Replace empty coordinates by interpolating
imputed_coords = []
for coord in polygon_coords:
    if coord is None:
        imputed_y = interp_func(coord[0])
        imputed_coords.append((coord[0], imputed_y))
    else:
        imputed_coords.append(coord)

# Create a new polygon with the replaced coordinates
imputed_polygon = Polygon(imputed_coords)

# Print the result
print(imputed_polygon)
Polygon(imputed_polygon) # This line creates a polygon from the imputed coordinates

# 3. Inward

In [ ]:
from shapely.geometry import Polygon, LineString

# Coordinates of an irregular polygon
polygon_coords = imputed_coords

# Create an irregular polygon
polygon = Polygon(polygon_coords)

# Distance to move inward
distance = 500  # unit: um (micrometers)

# Create line segments representing the boundary of the irregular polygon
boundary_lines = []
for i in range(len(polygon_coords)):
    start_point = tuple(polygon_coords[i])
    end_point = tuple(polygon_coords[(i + 1) % len(polygon_coords)])
    line = LineString([start_point, end_point])
    boundary_lines.append(line)

# Create new line segments moved inward
new_lines = []
for line in boundary_lines:
    # Calculate the normal vector of the line segment
    normal_vector = (line.coords[1][1] - line.coords[0][1], line.coords[0][0] - line.coords[1][0])

    # Calculate the magnitude of the normal vector
    norm = (normal_vector[0] ** 2 + normal_vector[1] ** 2) ** 0.5

    # Skip if the magnitude of the normal vector is zero
    if norm == 0:
        continue

    # Convert the normal vector to a unit vector
    unit_normal_vector = (normal_vector[0] / norm, normal_vector[1] / norm)

    # Create new line segments moved inward
    new_start_point = (line.coords[0][0]-distance * unit_normal_vector[0], line.coords[0][1] - distance * unit_normal_vector[1])
    new_end_point = (line.coords[1][0]-distance * unit_normal_vector[0], line.coords[1][1] - distance * unit_normal_vector[1])
    new_line = LineString([new_start_point, new_end_point])
    new_lines.append(new_line)

# Create a new polygon with the inwardly moved coordinates
new_polygon_coords_inward = []
for line in new_lines:
    new_polygon_coords_inward.append(line.coords[0])
new_polygon = Polygon(new_polygon_coords_inward)

# Print the results
print(new_polygon_coords_inward)


# 4. Outward

In [ ]:
from shapely.geometry import Polygon, LineString

# Coordinates of an irregular polygon
polygon_coords = imputed_coords

# Create an irregular polygon
polygon = Polygon(polygon_coords)

# Distance to move outward
distance = 500  # unit: um (micrometers)

# Create line segments representing the boundary of the irregular polygon
boundary_lines = []
for i in range(len(polygon_coords)):
    start_point = tuple(polygon_coords[i])
    end_point = tuple(polygon_coords[(i + 1) % len(polygon_coords)])
    line = LineString([start_point, end_point])
    boundary_lines.append(line)

# Create new line segments moved outward
new_lines = []
for line in boundary_lines:
    # Calculate the normal vector of the line segment
    normal_vector = (line.coords[1][1] - line.coords[0][1], line.coords[0][0] - line.coords[1][0])

    # Calculate the magnitude of the normal vector
    norm = (normal_vector[0] ** 2 + normal_vector[1] ** 2) ** 0.5

    # Skip if the magnitude of the normal vector is zero
    if norm == 0:
        continue

    # Convert the normal vector to a unit vector
    unit_normal_vector = (normal_vector[0] / norm, normal_vector[1] / norm)

    # Create new line segments moved outward
    new_start_point = (line.coords[0][0] + distance * unit_normal_vector[0], line.coords[0][1] + distance * unit_normal_vector[1])
    new_end_point = (line.coords[1][0] + distance * unit_normal_vector[1], line.coords[1][1] + distance * unit_normal_vector[1])
    new_line = LineString([new_start_point, new_end_point])
    new_lines.append(new_line)

# Create a new polygon with the outwardly moved coordinates
new_polygon_coords_outward = []
for line in new_lines:
    new_polygon_coords_outward.append(line.coords[0])
new_polygon = Polygon(new_polygon_coords_outward)

# Print the results
print(new_polygon_coords_outward)


# 5. Imputation

In [ ]:
polygon_inner=Polygon(new_polygon_coords_inward)
outer_buffer_distance =
inner_buffer_distance = 
outer_buffered_polygon = polygon_inner.buffer(outer_buffer_distance, join_style=1)
smoothed_polygon_inward = outer_buffered_polygon.buffer(inner_buffer_distance, join_style=1)
smoothed_polygon_inward
smoothed_polygon_inward

In [ ]:
polygon_outer=Polygon(new_polygon_coords_outward)
outer_buffer_distance =
inner_buffer_distance = 
outer_buffered_polygon = polygon_outer.buffer(outer_buffer_distance, join_style=1)
smoothed_polygon_outer = outer_buffered_polygon.buffer(inner_buffer_distance, join_style=1)
smoothed_polygon_outer
smoothed_polygon_outer

In [ ]:
polygon_border=Polygon(imputed_coords)
outer_buffer_distance = 
inner_buffer_distance = 
outer_buffered_polygon = polygon_outer.buffer(outer_buffer_distance, join_style=1)
smoothed_polygon_border = outer_buffered_polygon.buffer(inner_buffer_distance, join_style=1)
smoothed_polygon_border

In [ ]:

border_smoothed_coords = smoothed_polygon_border.exterior.coords[:]
boundary_border = Path(border_smoothed_coords)
is_inside =  np.array([not boundary_border.contains_point(coord) for coord in coords])
is_inside = [boundary.contains_point(coord) for coord in coords]
subset_adata = adata[is_inside].copy()
subset_adata.obs['Tumor']="Tumor"
#sq.pl.spatial_scatter(subset_adata, color="Level0",alpha=0.3, size=2,save='spatial_border.pdf')

In [ ]:
inward_smoothed_coords = smoothed_polygon_inward.exterior.coords[:]
boundary_inward = Path(inward_smoothed_coords)
is_inside2 =  np.array([not boundary_inward.contains_point(coord) for coord in coords])
is_inside2 = [boundary_inward.contains_point(coords) for coords in coords]
subset_inward = adata[is_inside2].copy()
subset_inward.obs['Tumor_core']="Core"
#sq.pl.spatial_scatter(subset_inward, color="Level0",img_alpha=0,alpha=0.3, size=2)

In [ ]:
outward_smoothed_coords = smoothed_polygon_outer.exterior.coords[:]
boundary_outward = Path(outward_smoothed_coords)
is_inside3 = [boundary_outward.contains_point(coords) for coords in coords]
subset_outward = adata[is_inside3].copy()
subset_outward.obs['Tumor_outward']="Outward"
#sq.pl.spatial_scatter(subset_outward, color="Level0",img_alpha=0,alpha=0.3, size=2)

# 6. Compartment

In [ ]:
subset_outward.obs = pd.merge(subset_outward.obs, subset_adata.obs[['cell_id','Tumor']], on='cell_id', how='left')
subset_outward.obs = pd.merge(subset_outward.obs, subset_inward.obs[['cell_id','Tumor_core']], on='cell_id', how='left')
subset_outward.var_names_make_unique()
subset_outward.obs.index = subset_outward.obs.index.astype(str)
subset_outward.var.index = subset_outward.var.index.astype(str)

In [ ]:
subset_outward.obs['Core_margin'] = subset_outward.obs['Tumor_core'].apply(lambda x: 'Core' if x == 'Core' else 'Invasive_margin')


In [ ]:
adata.obs = pd.merge(adata.obs, subset_outward.obs[['cell_id','Core_margin']], on='cell_id', how='left')
adata.obs = pd.merge(adata.obs, subset_adata.obs[['cell_id','Tumor']], on='cell_id', how='left')
adata.obs = pd.merge(adata.obs, subset_inward.obs[['cell_id','Tumor_core']], on='cell_id', how='left')


In [ ]:
adata.obs['Margin'] = pd.np.where(adata.obs['Core_margin'] == 'Core', 'Core',
                      pd.np.where(adata.obs['Tumor'] == 'Tumor', 'Inner_margin',
                                  pd.np.where(adata.obs['Core_margin'] == 'Invasive_margin', 'Outer_margin','Adjacent_tissue')))